In [1]:
import pandas as pd
import plotly.express as px
import streamlit as st
from gsheetsdb import connect
from google.oauth2 import service_account

In [2]:
# Create a connection object.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"],
    scopes=[
        "https://www.googleapis.com/auth/spreadsheets",
    ],
)

In [3]:
with connect(credentials=credentials) as conn:
    sheet_url = st.secrets["geolocations_url"]
    df_geoloc = conn.execute(f'SELECT * FROM "{sheet_url}"', headers=1).fetchall()
    df_geoloc = pd.DataFrame.from_dict(data=df_geoloc)

    sheet_url = st.secrets["sets_url"]
    df_sets = conn.execute(f'SELECT * FROM "{sheet_url}"', headers=1).fetchall()
    df_sets = pd.DataFrame.from_dict(data=df_sets)

    df = df_sets.merge(df_geoloc, on=["Venue", "Area"], how="left")

    df["latitude"] = pd.to_numeric(df.Latitude, errors="coerce")
    df["longitude"] = pd.to_numeric(df.Longitude, errors="coerce")
    df = df.drop(columns=["Comments", "Payments", "Latitude", "Longitude"])
    df.Date = pd.to_datetime(
        df.Date,
        format="%Y-%m-%d",
        errors="coerce",
    )
    df = df.sort_values(by="Date").reset_index(drop=True)
    df["SetNo"] = df.index + 1
    # df = df.set_index("SetNo")
    df.index = df.Date

    for col in [
        "Event",
        "Organizer",
        # "EventType",
        "VenueFullName",
    ]:
        df[col] = df[col].astype("category")

In [4]:
df

,Date,Event,Organizer,Venue,Area,EventType,VenueFullName,latitude,longitude,SetNo
Date,,,,,,,,,,
2014-12-02,2014-12-02,Latin Tuesdays with DDLL,Danza De La Liga,Red Bar,Koregaon Park,Social,"Red Bar, Koregaon Park",18.540159,73.908260,1
2014-12-16,2014-12-16,Latin Tuesdays with DDLL,Danza De La Liga,Red Bar,Koregaon Park,Social,"Red Bar, Koregaon Park",18.540159,73.908260,2
2014-12-30,2014-12-30,Latin Tuesdays with DDLL,Danza De La Liga,Red Bar,Koregaon Park,Social,"Red Bar, Koregaon Park",18.540159,73.908260,3
2015-01-13,2015-01-13,Latin Tuesdays with DDLL,Danza De La Liga,Red Bar,Koregaon Park,Social,"Red Bar, Koregaon Park",18.540159,73.908260,4
2015-03-15,2015-03-15,Sunday Night with Salsa,enMotion,The Flying Saucer Skybar,Viman Nagar,Social,"The Flying Saucer Skybar, Viman Nagar",18.568865,73.908304,5
...,...,...,...,...,...,...,...,...,...,...
2023-05-18,2023-05-18,Kolkata Salsa Weekend,Hitesh Salsawala,Offbeat CCU,Kolkata,Congress,"Offbeat CCU, Kolkata",22.533211,88.395724,182
2023-05-19,2023-05-19,Kolkata Salsa Weekend,Hitesh Salsawala,Offbeat CCU,Kolkata,Congress,"Offbeat CCU, Kolkata",22.533211,88.395724,183
2023-05-20,2023-05-20,Kolkata Salsa Weekend,Hitesh Salsawala,Offbeat CCU,Kolkata,Congress,"Offbeat CCU, Kolkata",22.533211,88.395724,184


In [16]:
dff = (
    df.assign(
        YearMonth=lambda x: x.index.strftime("%Y-%m"),
        # MonthYear=lambda x: x.index.strftime("%m-%Y"),
    )
    .groupby(["YearMonth", "EventType"])
    .SetNo.count()
    .reset_index(drop=False)
)
dff

,YearMonth,EventType,SetNo
0,2014-12,Social,3
1,2015-01,Social,1
2,2015-03,Social,2
3,2015-05,Social,1
4,2015-05,Weekender,1
...,...,...,...
72,2022-03,Social,1
73,2022-04,Congress,4
74,2023-04,Social,3
75,2023-05,Congress,4


In [7]:
fig = px.bar(dff, x="MonthYear", y="SetNo")
fig.show()

In [ ]:
dff = (
    df.resample("M")
    .SetNo.count()
    .reset_index(drop=False)
    .assign(MonthYear=lambda x: x.Date.dt.strftime("%b %Y"))
    .drop(columns=["Date"])
    .rename(columns={"SetNo": "Sets Count"})
    # .plot.bar(x="MonthYear", y="Sets Count")
)

In [ ]:
fig = px.bar(dff, x="MonthYear", y="Sets Count")
# st.plotly_chart(fig, use_container_width=True)
fig.show()